In [ ]:
# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                                             library / module imports                                            ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib import patches
import ipywidgets as widgets
from IPython.display import display, clear_output

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                                        global variables / initialization                                        ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
df = pd.DataFrame()
pressure_df = pd.DataFrame()
step_changes = pd.DataFrame()
step_times = []
step_labels = []
unique_steps = []
step_df = pd.DataFrame()
steady_frac = 0.2
slope_tol_s = 10.0
amp_tol = 50.0
min_points = 5
color_map = {
    'Wait Before Start': '#dab1da',
    'Predose Pump':      '#f0d3ef',
    'N2Overlap':         '#c9d9ec',
    'PumpOverlap':       '#dceaf7',
    'Open Time':         '#b9e0c6',
    'Hold':              '#d5edd8',
    'Pump Purge':        '#fbd2a5',
    'Purge':             '#ffe0cb'
}

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                                      file loader / max pressure logic                                           ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
file_path_text = widgets.Text(value='', placeholder='Paste JSON file path here', description='File Path:',
                               style={'description_width':'initial'}, layout=widgets.Layout(width='800px'))
load_button = widgets.Button(description="Load File", button_style='primary', icon='check', layout=widgets.Layout(width='150px'))
load_status = widgets.Label(value='')

def load_file(button):
    global df, pressure_df, step_changes, step_times, step_labels, unique_steps, step_df
    file_path = file_path_text.value.strip()
    if not file_path:
        load_status.value = "Please enter a file path."
        return
    try:
        with open(file_path,'r') as file:
            data = [json.loads(line) for line in file]
        for entry in data:
            if isinstance(entry.get("payload"), str):
                entry["payload"] = json.loads(entry["payload"])
        df = pd.json_normalize(data)
        df["payload.TimeElapsed"] = pd.to_numeric(df["payload.TimeElapsed"], errors="coerce")
        df["payload.Pressure"] = pd.to_numeric(df["payload.Pressure"], errors="coerce")
        df["payload.CurrentCycle"] = pd.to_numeric(df["payload.CurrentCycle"], errors="coerce")
        pressure_df = df[df["type"]=="pressure"].copy()
        
        step_changes = df[df["payload.CurrentStep"].notna()].copy()
        step_changes["PreviousStep"] = step_changes["payload.CurrentStep"].shift()
        step_changes = step_changes[step_changes["payload.CurrentStep"]!=step_changes["PreviousStep"]]
        step_times = step_changes["payload.TimeElapsed"].tolist()
        step_times.append(df["payload.TimeElapsed"].max())
        step_labels = step_changes["payload.CurrentStep"].tolist()
        unique_steps = list(dict.fromkeys(step_labels)) # could just hardcode this since color_map is hardcoded

        step_cycles = step_changes["payload.CurrentCycle"].astype(int).tolist()
        step_df = pd.DataFrame({
            "Cycle": step_cycles,
            "Step": step_labels,
            "Start Time (ms)": step_times[:-1],
            "End Time (ms)": step_times[1:]
        })
        step_df["Duration (ms)"] = step_df["End Time (ms)"]-step_df["Start Time (ms)"]
        step_df["Duration (s)"] = step_df["Duration (ms)"]/1000

        # max pressure can be calculated once when loading the file because it doesn't depend on any adjustable parameters,
        # unlike steady-state pressure, which depends on user-defined criteria like steady fraction, slope tolerance, etc.
        max_pressures = []
        for i in range(len(step_df)):
            start, end = step_df.loc[i,"Start Time (ms)"], step_df.loc[i,"End Time (ms)"]
            time_range_bool = (df["payload.TimeElapsed"]>=start) & (df["payload.TimeElapsed"]<end)
            max_pressures.append(df.loc[time_range_bool,"payload.Pressure"].max())
        step_df["Max Pressure (mTorr)"] = max_pressures
        # but we still want to calculate an initial steady-state profile
        compute_steady_pressures_and_slopes()

        load_status.value = f"Loaded file: {file_path}"
        update_all_tabs()
    except Exception as err:
        load_status.value = f"{err}"

load_button.on_click(load_file)

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                                                pressure plotter                                                 ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
def plot_pressure(cycle:int = None):
    fig, ax = plt.subplots(figsize=(15,5))
    ax.plot(pressure_df["payload.TimeElapsed"], pressure_df["payload.Pressure"], color='black', linewidth=2, label="Pressure")
    for i in range(len(step_labels)):
        start, stop = step_times[i], step_times[i+1]
        label = step_labels[i]
        color = color_map.get(label, '#cccccc')
        ax.add_patch(patches.Rectangle((start, ax.get_ylim()[0]), stop-start, ax.get_ylim()[1]-ax.get_ylim()[0], color=color, alpha=1))
    legend_handles = [patches.Patch(color=color_map[step], label=step) for step in unique_steps]
    ax.legend(handles=legend_handles, title="Step", bbox_to_anchor=(1.01,1), loc='upper left')
    ax.set_xlabel("Time Elapsed [ms]")
    ax.set_ylabel("Pressure [mTorr]")

    if cycle is None:
        ax.set_title("Pressure vs Time Elapsed")
        cycle_changes = df[df["payload.CurrentCycle"].notna()].copy()
        cycle_changes["PreviousCycle"] = cycle_changes["payload.CurrentCycle"].shift()
        cycle_changes = cycle_changes[cycle_changes["payload.CurrentCycle"] != cycle_changes["PreviousCycle"]]
        cycle_changes["payload.TimeElapsed"] = pd.to_numeric(cycle_changes["payload.TimeElapsed"], errors="coerce")
        for time in cycle_changes["payload.TimeElapsed"]:
            ax.axvline(x=time, color='red', linestyle='--', linewidth=2, alpha=1)
        plt.xlim([0, df["payload.TimeElapsed"].max()])
    else:
        cycle_start = df[df["payload.CurrentCycle"]==cycle]["payload.TimeElapsed"].min()
        cycle_end = df[df["payload.CurrentCycle"]==cycle+1]["payload.TimeElapsed"].min()
        if pd.isna(cycle_end):
            cycle_end = df["payload.TimeElapsed"].max()
        ax.set_title(f"Pressure vs Time Elapsed (Cycle {cycle})")
        plt.xlim([cycle_start, cycle_end])

    plt.tight_layout()
    plt.rcParams['figure.dpi'] = 200 
    plt.show()

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                                              steady-state calculations                                          ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
def steady_pressure_for_window(times_ms, pressures):
    valid_points_array = np.isfinite(times_ms) & np.isfinite(pressures) # remove NaN values
    
    if not np.any(valid_points_array): 
        return np.nan, np.nan

    # use only valid points
    valid_times_ms = np.asarray(times_ms[valid_points_array], dtype=float)
    valid_pressures = np.asarray(pressures[valid_points_array], dtype=float)

    # need at least 2 points to compute a slope
    if valid_times_ms.size < 2: 
        return np.nan, np.nan

    # sort by increasing time for slope calculations
    order = np.argsort(valid_times_ms)
    valid_times_ms, valid_pressures = valid_times_ms[order], valid_pressures[order]

    # if multiple pressure values have the same timestamp (don't think this would ever happen anyways), average them
    unique_time, inv = np.unique(valid_times_ms, return_inverse=True)
    sums = np.bincount(inv, weights=valid_pressures)
    counts = np.bincount(inv)
    valid_pressures = sums / counts
    valid_times_ms = unique_time
    
    times_s = valid_times_ms / 1000.0
    num_points = valid_times_ms.size
    
    start_index = max(int(np.floor(num_points*(1-steady_frac))),0)
    steady_times, steady_pressures = times_s[start_index:], valid_pressures[start_index:]
    
    if steady_times.size < min_points:
        return np.nan, np.nan
    
    duration_s = steady_times[-1]-steady_times[0]
    
    if duration_s <= 0:
        return np.nan, np.nan

    if steady_times.size >= 3:
        slope_s = np.polyfit(steady_times, steady_pressures, 1)[0]
    else:
        slope_s = (steady_pressures[-1]-steady_pressures[0])/duration_s
    
    amp = float(np.nanmax(steady_pressures)-np.nanmin(steady_pressures))
    is_steady = (abs(slope_s)<=slope_tol_s) and (amp<=amp_tol)
    return (float(np.nanmean(steady_pressures)) if is_steady else np.nan, slope_s)
    

def compute_steady_pressures_and_slopes():
    global step_df
    if step_df.empty or df.empty:
        return

    steady_pressures, slopes = [], []

    for i in range(len(step_df)):
        start, end = step_df.loc[i, "Start Time (ms)"], step_df.loc[i, "End Time (ms)"]
        time_range_bool = (df["payload.TimeElapsed"] >= start) & (df["payload.TimeElapsed"] < end)
        times = df.loc[time_range_bool, "payload.TimeElapsed"].to_numpy()
        pressures = df.loc[time_range_bool, "payload.Pressure"].to_numpy()
        steady_p, slope_s = steady_pressure_for_window(times, pressures)
        steady_pressures.append(steady_p)
        slopes.append(slope_s)

    step_df["Steady Pressure (mTorr)"] = steady_pressures
    step_df["Slope (mTorr/s)"] = slopes

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                                                 helper functions                                                ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
def update_cycle_dropdown():
    if df.empty:
        cycle_dropdown.options = ['All']
        cycle_dropdown.value = 'All'
        return

    # temporarily remove observer to prevent update_pressure_plot() from running twice when initializing dropdown options
    try:
        cycle_dropdown.unobserve(update_pressure_plot, names='value')
    except:
        pass

    # update options
    cycle_dropdown.options = ['All'] + sorted(df['payload.CurrentCycle'].dropna().unique().astype(int).tolist())[1:]
    cycle_dropdown.value = 'All'

    # re-add observer
    cycle_dropdown.observe(update_pressure_plot, names='value')
    

def update_step_selectors():
    options = ['All'] + step_df['Step'].dropna().unique().tolist() if not step_df.empty else ['All']
    max_step_selector.options = options
    steady_step_selector.options = options

    if 'All' in max_step_selector.options:
        max_step_selector.value = ['All']
    if 'All' in steady_step_selector.options:
        steady_step_selector.value = ['All']


# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                                                      widgets                                                    ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
cycle_dropdown = widgets.Dropdown(description='Choose Cycle:', style={'description_width':'initial'}, layout=widgets.Layout(width='250px'))


max_step_selector = widgets.SelectMultiple(
    options=['All'],          
    value=['All'],            
    description='Choose Step(s):',
    disabled=False,
    style={'description_width':'initial'},
    rows=9,
    layout=widgets.Layout(width='300px')
)

steady_step_selector = widgets.SelectMultiple(
    options=['All'],
    value=['All'],
    description='Choose Step(s):',
    disabled=False,
    style={'description_width':'initial'},
    rows=9,
    layout=widgets.Layout(width='300px', height='120px')
)

pressure_output = widgets.Output()
max_pressure_output = widgets.Output()
steady_state_output = widgets.Output()
missing_pressure_output = widgets.Output()
film_thickness_output = widgets.Output()
pressure_loading = widgets.Label(value='')

# steady-state parameters widgets
steady_frac_widget = widgets.BoundedFloatText(
    value=steady_frac, min=0.01, max=1.0, step=0.01,
    description='Steady Fraction:', style={'description_width':'initial'},
    layout=widgets.Layout(width='250px')
)

slope_tol_widget = widgets.BoundedFloatText(
    value=slope_tol_s, min=0.0, step=0.1,
    description='Slope Tolerance (mTorr/s):', style={'description_width':'initial'},
    layout=widgets.Layout(width='250px')
)

amp_tol_widget = widgets.BoundedFloatText(
    value=amp_tol, min=0.0, step=1.0,
    description='Peak-to-Peak Amp (mTorr):', style={'description_width':'initial'},
    layout=widgets.Layout(width='250px')
)

min_points_widget = widgets.BoundedIntText(
    value=min_points, min=2, step=1,
    description='Minimum Points:', style={'description_width':'initial'},
    layout=widgets.Layout(width='250px')
)

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                                             widget update functions                                             ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
def update_all_tabs(change=None):
    update_cycle_dropdown()
    update_step_selectors()
    update_pressure_plot({'new': cycle_dropdown.value})
    update_max_pressure()
    update_steady_state()
    update_missing_pressure()
    update_film_thickness()

def update_pressure_plot(change):
    cycle_selection = change['new']
    pressure_loading.value = f'Loading cycle {cycle_selection}...' if cycle_selection!='All' else 'Loading all cycles...'
    with pressure_output:
        clear_output(wait=True)
        if not pressure_df.empty:
            plot_pressure(cycle_selection if cycle_selection!='All' else None)
    pressure_loading.value=''

def update_max_pressure(change=None):
    selected_steps = max_step_selector.value
    with max_pressure_output:
        clear_output(wait=True)
        if step_df.empty:
            print("No data")
            return
        df_show = step_df if 'All' in selected_steps else step_df[step_df['Step'].isin(selected_steps)]

        plt.figure(figsize=(6,6))
        for step in df_show['Step'].unique():
            subset = df_show[df_show['Step']==step]
            color = color_map.get(step, '#00aa33')
            plt.scatter(subset['Cycle'], subset['Max Pressure (mTorr)'], color=color,
                        edgecolor='black', linewidth=0.7, label=step)
        plt.title('Max Pressure by Step')
        plt.xlabel('Cycle')
        plt.ylabel('Max Pressure (mTorr)')
        plt.legend(title='Step', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlim(step_df["Cycle"].min(), step_df["Cycle"].max())
        plt.xticks(range(step_df["Cycle"].min(), step_df["Cycle"].max() + 1))
        plt.gca().minorticks_on()
        plt.grid(False)
        plt.show()

        display(df_show)

def update_steady_state(change=None):
    selected_steps = steady_step_selector.value
    with steady_state_output:
        clear_output(wait=True)
        if step_df.empty:
            print("No data")
            return
        df_show = step_df if 'All' in selected_steps else step_df[step_df['Step'].isin(selected_steps)]

        plt.figure(figsize=(6,6))
        for step in df_show['Step'].unique():
            subset = df_show[df_show['Step']==step]
            color = color_map.get(step, '#00aa33')
            plt.scatter(subset['Cycle'], subset['Steady Pressure (mTorr)'], color=color,
                        edgecolor='black', linewidth=0.7, label=step)
        plt.title('Steady-State Pressure by Step')
        plt.xlabel('Cycle')
        plt.ylabel('Steady Pressure (mTorr)')
        plt.legend(title='Step', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.xlim(step_df["Cycle"].min(), step_df["Cycle"].max())
        plt.xticks(range(step_df["Cycle"].min(), step_df["Cycle"].max() + 1))
        plt.gca().minorticks_on()
        plt.grid(False)
        plt.show()

        display(df_show)

cycle_dropdown.observe(update_pressure_plot, names='value')
max_step_selector.observe(update_max_pressure, names='value')
steady_step_selector.observe(update_steady_state, names='value')


def update_steady_params(change):
    global steady_frac, slope_tol_s, amp_tol, min_points
    steady_frac = steady_frac_widget.value
    slope_tol_s = slope_tol_widget.value
    amp_tol = amp_tol_widget.value
    min_points = min_points_widget.value

    # recompute steady pressure and slope
    compute_steady_pressures_and_slopes()
    update_steady_state() # refresh plot

for w in [steady_frac_widget, slope_tol_widget, amp_tol_widget, min_points_widget]:
    w.observe(update_steady_params, names='value')

def update_missing_pressure():
    missing_pressure_output.clear_output(wait=True)
    with missing_pressure_output:
        if pressure_df.empty: print("No pressure data loaded yet.")
        else:
            nan_count = pressure_df["payload.Pressure"].isna().sum()
            print(f"The number of missing pressure packets from the original JSON file is {nan_count}.")

def update_film_thickness():
    film_thickness_output.clear_output(wait=True)
    with film_thickness_output:
        print("Film Thickness placeholder")

def update_step_selectors():
    options = ['All'] + step_df['Step'].dropna().unique().tolist() if not step_df.empty else ['All']
    max_step_selector.options = options
    steady_step_selector.options = options
    
    if 'All' in max_step_selector.options:
        max_step_selector.value = ['All']
    if 'All' in steady_step_selector.options:
        steady_step_selector.value = ['All']

# ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
# ┃                         tab descriptions, misc. info tab, descriptions, UI display, etc.                        ┃    
# ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
steady_info_text = """
<b><u>Steady-State Tab Information</u></b><br>

The <i>Steady-State</i> tab uses four criteria to determine if a step has reached steady state:<br>

1. <b><u>Steady Fraction</u></b>:<br>
   - The fraction of a step's final points to use for the steady-state calculation.<br>
   - Example: 0.2 means only the last 20% of a step's points are analyzed.<br>

2. <b><u>Slope Tolerance</u></b>:<br> 
   - The maximum allowed slope of a least-squares line fitted to the selected points.<br>
   - A slope exceeding this value has not reached steady state (dP/dt≈0).<br>

3. <b><u>Peak-to-Peak Amplitude Tolerance</u></b>:<br>
   - The maximum difference between the highest and lowest points in the selected fraction.<br>
   - Large fluctuations prevent the step from being considered steady, even if the slope is small.<br>

4. <b><u>Minimum Number of Points in Step</u></b>:<br>
   - A step must contain at least the designated minimum number of points with recorded time values to
ensure an accurate representation of the step's behavior.<br>
<br>
Note: For <i>Max Plot</i> and <i>Steady-State Plot</i>, multiple steps can be selected with shift and/or
ctrl (or command) pressed and mouse clicks or arrow keys.

"""

steady_info_output = widgets.Output()
with steady_info_output:
    display(widgets.HTML(steady_info_text))

# UI display
display(widgets.VBox([file_path_text, load_button, load_status]))

# descriptions
pressure_desc = widgets.HTML("<b>Pressure Plot:</b> Shows pressure vs. time for the selected cycle by step.")
max_pressure_desc = widgets.HTML("<b>Max Pressure:</b> Shows max pressure vs. cycle by step.")
steady_state_desc = widgets.HTML("<b>Steady-State Pressure:</b> Shows steady-state pressure vs. cycle by step.")
missing_pressure_desc = widgets.HTML("<b>Missing Pressure Test:</b> Counts missing pressure data from the loaded JSON file.")
film_thickness_desc = widgets.HTML("<b>Film Thickness:</b> Placeholder for film thickness plots or data.")

steady_widgets_box = widgets.HBox([steady_frac_widget, slope_tol_widget, amp_tol_widget, min_points_widget])

tab = widgets.Tab(children=[
    widgets.VBox([pressure_desc, cycle_dropdown, pressure_loading, pressure_output]),
    widgets.VBox([max_pressure_desc, max_step_selector, max_pressure_output]),
    widgets.VBox([steady_state_desc, steady_widgets_box, steady_step_selector, steady_state_output]),
    widgets.VBox([missing_pressure_desc, missing_pressure_output]),
    widgets.VBox([film_thickness_desc, film_thickness_output]),
    steady_info_output
])

tab.set_title(0, 'Pressure Plot')
tab.set_title(1, 'Max Plot')
tab.set_title(2, 'Steady-State Plot')
tab.set_title(3, 'Missing Data Test')
tab.set_title(4, 'Film Thickness')
tab.set_title(5, 'Misc. Info')

display(tab)

In [ ]:
# df

In [ ]:
# pressure_df

In [ ]:
# step_df